In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [3]:
train_data = train_datagen.flow_from_directory('C:/Users/DNIN/Desktop/machine learning tutorial/train',target_size = (224,224),batch_size = 32,class_mode = 'categorical')

Found 3200 images belonging to 4 classes.


In [4]:
test_data = train_datagen.flow_from_directory('C:/Users/DNIN/Desktop/machine learning tutorial/test',target_size = (224,224),batch_size = 32,class_mode = 'categorical')

Found 800 images belonging to 4 classes.


In [5]:
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model

In [6]:

Image_size = [224, 224]

efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=Image_size + [3])
for layer in efficientnet_model.layers:
  layer.trainable = False


In [7]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api import activations
y = tf.keras.layers.Flatten()(efficientnet_model.output)

num_classes = 4  # Adjust based on your dataset

final_layer = Dense(num_classes, activation='softmax')(y)

efficientnet_model = Model(inputs=efficientnet_model.input, outputs=final_layer)


In [8]:

efficientnet_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling (Rescaling)         │ (None, 224, 224, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 224, 224, 3)       │               7 │ rescaling[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ normalization[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 225, 225, 3)       │               0 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 112, 112, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 112, 112, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 112, 112, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 112, 112, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 112, 112, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 112, 112, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             288 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 4,300,455 (16.40 MB)

 Trainable params: 250,884 (980.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [9]:
loss_weight = 0.05
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],loss_weights = loss_weight)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

# Set up early stopping to monitor validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3,mode = 'max')

# Train the model with early stopping
efficientnet_model.fit(train_data, epochs=20, validation_data = test_data , callbacks=[early_stopping])

Epoch 1/100


C:\Users\DNIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.7914 - loss: 0.0492 - val_accuracy: 0.9000 - val_loss: 0.0333
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.9415 - loss: 0.0194 - val_accuracy: 0.8562 - val_loss: 0.0685
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.9449 - loss: 0.0185 - val_accuracy: 0.8475 - val_loss: 0.0977
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.9538 - loss: 0.0197 - val_accuracy: 0.8838 - val_loss: 0.0763


In [11]:
efficientnet_model.save('cnn.h5')

In [12]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [17]:
labels = ['0_normal','1_ulcerative_colitis','2_polyps','3_esophagitis']
img_path = 'C:/Users/DNIN/Desktop/machine learning tutorial/PXL_20240716_041341655~2.jpg'# Replace with your image path
#"C:\Users\DNIN\Desktop\machine learning tutorial\test\1_ulcerative_colitis\test_ulcer_ (1).jpg"
img = load_img(img_path, target_size=(224,224))
x = img_to_array(img)
x = preprocess_input(x)
preds = efficientnet_model.predict(np.array([x]))
preds

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


array([[2.6916661e-33, 1.6112051e-07, 3.5896212e-28, 9.9999988e-01]],
      dtype=float32)

In [18]:
labels[np.argmax(preds)]

'3_esophagitis'